In [16]:
import pandas as pd
import numpy as np

In [17]:
ratings = pd.read_csv('game_ratings.csv.gz', encoding='utf-8', compression='gzip', index_col=0)

In [18]:
ratings.drop(ratings.columns[ ratings.notnull().sum() < 2 ], axis=1, inplace=True)

In [19]:
ratings.head()


,AceAceBaby,AdmiralACF,agreen53,ALSB,Altasia,Altoid Guy,anatana,AnotherHorrorFan,Armadillo Al,auroramu,...,mithras,Priesst,Shock trooper,chekkie,CletusVanDamme,everettlane,Gadgetx,Jedimaster25,quizox,Truthsayer
175223,8.0,8.0,8.0,7.0,8.0,8.0,9.0,8.5,7.0,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40834,NaN,NaN,NaN,NaN,NaN,8.0,NaN,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178900,NaN,5.0,NaN,NaN,NaN,NaN,9.0,9.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104162,NaN,6.0,6.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36218,5.0,NaN,3.0,NaN,NaN,8.0,NaN,9.0,9.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
mean = ratings.mean(axis=1)
pd.DataFrame(mean).head()

,0
175223,7.622418
40834,7.721869
178900,7.904056
104162,7.603839
36218,7.605531


In [21]:
normalized = ratings.sub(mean, axis=0)
normalized.head()

,AceAceBaby,AdmiralACF,agreen53,ALSB,Altasia,Altoid Guy,anatana,AnotherHorrorFan,Armadillo Al,auroramu,...,mithras,Priesst,Shock trooper,chekkie,CletusVanDamme,everettlane,Gadgetx,Jedimaster25,quizox,Truthsayer
175223,0.377582,0.377582,0.377582,-0.622418,0.377582,0.377582,1.377582,0.877582,-0.622418,-1.122418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40834,NaN,NaN,NaN,NaN,NaN,0.278131,NaN,1.278131,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178900,NaN,-2.904056,NaN,NaN,NaN,NaN,1.095944,1.095944,1.095944,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104162,NaN,-1.603839,-1.603839,NaN,NaN,NaN,NaN,0.396161,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36218,-2.605531,NaN,-4.605531,NaN,NaN,0.394469,NaN,1.394469,1.394469,-0.605531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
sqrt_sum_squares = normalized.pow(2).sum(axis=1).pow(0.5)

In [48]:
import sys

sim = pd.DataFrame()

def similarity(i, j):
    nom = (normalized.loc[i]*normalized.loc[j]).sum()
    denom = sqrt_sum_squares.loc[i]*sqrt_sum_squares.loc[j]
    return nom/denom

sys.stdout.write('=' * normalized.index.shape[0])
sys.stdout.write('\n')
for i_index, i in enumerate(normalized.index.values):
    for j_index, j in enumerate(normalized.index.values):
        if (i_index <= j_index):
            sim.loc[i,j] = similarity(i,j)
        else:
            sim.loc[i,j] = sim.loc[j,i]

    sys.stdout.write('.')
    sys.stdout.flush()

..................................................................................................................................................................................................................................................................................................

In [49]:
sim.to_csv('simularity.csv.gz', compression='gzip')